In [ ]:
import numpy as np
from phi.flow import *

# N is the number of grid points
N = 128

# DX is the grid spacing
DX = 2.0 / N

# STEPS is the number of time steps
STEPS = 100

# DT is the time step size
DT = 2.0 / STEPS

# NU is the viscosity coefficient, defined here as 0.01 divided by the product of N and pi
NU = 0.01 / (N * np.pi)

# initialization of velocities, cell centers of a CenteredGrid have DX/2 offsets for linspace()
#INITIAL_NUMPY = np.asarray( [-np.sin(np.pi * x) + np.cos(x) + np.cos(2 * x) for x in np.linspace(-1+DX/2,1-DX/2,N)] ) # 1D numpy array

INITIAL_NUMPY = np.zeros(N)

INITIAL = math.tensor(INITIAL_NUMPY, spatial('x') ) # convert to phiflow tensor

#velocity = CenteredGrid(INITIAL, extrapolation.PERIODIC, x=N, bounds=Box[1]((-1,1))) # initial velocity
velocity = CenteredGrid(INITIAL, extrapolation.PERIODIC, x=N, bounds=Box['x', slice(-1,1)]) # initial velocity# initial velocity
velocities = [velocity] # list to store velocity at each time step

age = 0. # simulation time

# Number of sinusoidal functions
N_force = 2

# Wavenumbers
l_values = np.array([3, 4, 5, 6])

# Forcing
x = np.linspace(-1+DX/2,1-DX/2,N)  # Extract the single array from the tuple

for i in range(STEPS):
    # Initialize f
    f = np.zeros_like(x)

    # Add sinusoidal functions
    for _ in range(N_force):
        # Draw parameters
        A = np.random.uniform(-0.5, 0.5)
        ω = np.random.uniform(-0.4, 0.4)
        φ = np.random.uniform(0, 2*np.pi)
        l = np.random.choice(l_values)

        # Add sinusoidal function to f
        f += A * np.sin(ω * i * DT + 2*np.pi * l * x / 2 + φ)

    # Convert force to phiflow tensor
    force = math.tensor(f, spatial('x'))

    v1 = diffuse.explicit(velocities[-1], NU, DT) + force
    v2 = advect.semi_lagrangian(v1, v1, DT)
    age += DT
    velocities.append(v2)